In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [2]:
#import
df_matches = pd.read_csv("WorldCupMatches.csv")
df_players = pd.read_csv("WorldCupPlayers.csv")
df_world_cups = pd.read_csv("WorldCups.csv")

In [4]:
df_players.describe().T

,count,mean,std,min,25%,50%,75%,max
RoundID,37784.0,1.105647e+07,2.770144e+07,201.0,263.0,337.0,255931.0,97410600.0
MatchID,37784.0,6.362233e+07,1.123916e+08,25.0,1199.0,2216.0,97410003.0,300186515.0
Shirt Number,37784.0,1.072602e+01,6.960138e+00,0.0,5.0,11.0,17.0,23.0


In [5]:
df_matches.describe().T

,count,mean,std,min,25%,50%,75%,max
Year,852.0,1.985089e+03,2.244882e+01,1930.0,1970.00,1990.0,2002.00,2014.0
Home Team Goals,852.0,1.811033e+00,1.610255e+00,0.0,1.00,2.0,3.00,10.0
Away Team Goals,852.0,1.022300e+00,1.087573e+00,0.0,0.00,1.0,2.00,7.0
Attendance,850.0,4.516480e+04,2.348525e+04,2000.0,30000.00,41579.5,61374.50,173850.0
Half-time Home Goals,852.0,7.089202e-01,9.374141e-01,0.0,0.00,0.0,1.00,6.0
Half-time Away Goals,852.0,4.284038e-01,6.912519e-01,0.0,0.00,0.0,1.00,5.0
RoundID,852.0,1.066177e+07,2.729613e+07,201.0,262.00,337.0,249722.00,97410600.0
MatchID,852.0,6.134687e+07,1.110572e+08,25.0,1188.75,2191.0,43950059.25,300186515.0
